# Filtering log data with RePlay

This notebook presents the RePlay functionality for interaction log data filtering.<br>
You can conveniently filter your datasets, for example, leave only positive interactions, select the desired period of time or number of interactions, filter out users and items with low number of interactions.
<br>RePlay offers filtering functions:

* filter_by_min_count - filter out users or items with a few interactions
* filter_out_low_ratings - filter out low ratings/relevance values
* take_num_user_interactions - select required number of the first/last user's interactions
* take_num_days_of_global_hist - select required number of days from interaction log
* take_num_days_of_user_hist - select required number of days from each user's interaction log
* take_time_period - select interactions from required time period


In [ ]:
! pip install rs-datasets -q

In [1]:
import pandas as pd
from rs_datasets import MovieLens

from pyspark.sql import functions as sf
from pyspark.sql.types import TimestampType

from replay.utils.session_handler import State
from replay.utils.spark_utils import get_log_info, convert2spark, process_timestamp_column
from replay.preprocessing.filters import (
    filter_by_min_count,
    filter_out_low_ratings,
    take_num_user_interactions,
    take_num_days_of_user_hist,
    take_time_period,
    take_num_days_of_global_hist
)

In [3]:
spark = State().session
spark.sparkContext.setLogLevel('ERROR')
spark

## Get started

Download the dataset **MovieLens**

In [4]:
data = MovieLens("1m")
log = data.ratings

Use the `convert2spark` function to convert pandas.DataFrame to spark.DataFrame

In [5]:
log_spark = convert2spark(log)

Convert timestamp to appropriate format

In [6]:
log_spark = process_timestamp_column(log_spark, "timestamp")

In [7]:
log_spark.show(4)

+-------+-------+------+-------------------+
|user_id|item_id|rating|          timestamp|
+-------+-------+------+-------------------+
|      1|   1193|     5|2001-01-01 01:12:40|
|      1|    661|     3|2001-01-01 01:35:09|
|      1|    914|     3|2001-01-01 01:32:48|
|      1|   3408|     4|2001-01-01 01:04:35|
+-------+-------+------+-------------------+
only showing top 4 rows



## function filter_by_min_count()

Filter out users or items with number of interactions less than given threshold.

Parameters:

* `data_frame` - DataFrame with interactions between users and items

* `num_entries` - threshold for number of interactions
    
* `group_by` - column relative to which filtering occurs
    

The picture shows how filtering works using the example of a simple interaction log.<br>
Colored cells represent rating for given user and item. The color depends on the rating.

![title](img/filter_by_min_count.jpg)

Let's leave users who have `>= 50` interactions with items

In [8]:
log_filter = filter_by_min_count(log_spark, 
                                 num_entries=50,
                                 group_by="user_id")

13-Feb-23 15:50:09, replay, INFO: current threshold removes 0.0567261442358547% of data


Check minimum number of interactions per user

In [9]:
log_filter.groupBy("user_id").count().select(sf.min("count").alias("Minimum number of interactions per user's")).show()

+-----------------------------------------+
|Minimum number of interactions per user's|
+-----------------------------------------+
|                                       50|
+-----------------------------------------+



Number of users has decreased

In [10]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 943471, total users: 4297, total items: 3689


Let's leave items that `>= 100` users interacted with

In [11]:
log_filter = filter_by_min_count(log_spark, 
                                 num_entries=100,
                                 group_by="item_id")

13-Feb-23 15:50:18, replay, INFO: current threshold removes 0.05797188387626986% of data


Check minimum number of interactions per item

In [12]:
log_filter.groupBy("item_id").count().select(sf.min("count").alias("Minimum number of interactions per item's")).show()

+-----------------------------------------+
|Minimum number of interactions per item's|
+-----------------------------------------+
|                                      100|
+-----------------------------------------+



Number of items has decreased

In [13]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 942225, total users: 6040, total items: 2019


## function filter_out_low_ratings()

Filter out interactions with ratings/relevance less than given threshold.

Parameters:

* `data_frame` - DataFrame with interactions between users and items

* `value` - rating/relevance threshold

* `rating_column` - rating/relevance column
    

![title](img/filter_out_low_ratings.jpg)

Let's leave interactions with rating `>= 4`

In [14]:
log_filter = filter_out_low_ratings(log_spark,
                                    value=4,
                                    rating_column="rating")

Check minimum and maximum rating

In [15]:
log_filter.select(sf.min("rating").alias("Minimum rating")).show()
log_filter.select(sf.max("rating").alias("Maximum rating")).show()

+--------------+
|Minimum rating|
+--------------+
|             4|
+--------------+

+--------------+
|Maximum rating|
+--------------+
|             5|
+--------------+



The number of users and interactions has decreased

In [16]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 575281, total users: 6038, total items: 3533


## function take_num_user_interactions()

Select required number of the first/last user's interactions.

Parameters:


* `log` - DataFrame with interactions between users and items
    
* `num_interactions` - threshold value for the number of interactions for each user
    
* `first` - indicator, if the value is `True`, take first `num_interactions` for each user, if the value is `False`, take last `num_interactions` for each user
    
* `date_col` - date column name
    
* `user_col` - user id column name
    
* `item_col` - item id column name


![title](img/take_num_user_interactions.jpg)

Let's leave the last `2` interactions for each user

In [17]:
log_filter = take_num_user_interactions(log_spark, 
                                        num_interactions=2,
                                        first=False,
                                        user_col="user_id",
                                        item_col="item_id")

Check maximum number of interactions per user

In [19]:
log_filter.groupBy("user_id").count().select(sf.max("count").alias("Maximum number of interactions per user's")).show()

+-----------------------------------------+
|Maximum number of interactions per user's|
+-----------------------------------------+
|                                        2|
+-----------------------------------------+



Note that the number of users has not changed

In [20]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 12080, total users: 6040, total items: 2430


## function take_num_days_of_user_hist()

Select required number of days from each user's interaction log.

Parameters:

* `log` - DataFrame with interactions between users and items
    
* `days` - time interval in days
    
* `first` - indicator, if the value is `True`, take first days of each user's interactions, if the value is `False`, take last days of each user's interactions
    
* `date_col` - date column name
    
* `user_col` - user id column name

![title](img/take_num_days_of_user_hist.jpg)

Let's leave the last `week` of each user's interactions

In [21]:
log_filter = take_num_days_of_user_hist(log_spark, 
                                        days=7,
                                        first=False,
                                        date_col="timestamp",
                                        user_col="user_id")

Check maximum length of user history

In [23]:
(
    log_filter
    .groupBy("user_id")
    .agg(sf.datediff(sf.max("timestamp"), sf.min("timestamp")).alias("diff"))
    .select(sf.max("diff").alias("Maximum difference in days per user's"))
).show()

+-------------------------------------+
|Maximum difference in days per user's|
+-------------------------------------+
|                                    7|
+-------------------------------------+



The number of users has not changed, but the number of interactions has decreased

In [24]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 551897, total users: 6040, total items: 3599


## function take_num_days_of_global_hist()

Select required number of days from interaction log.

Parameters:

* `log` - DataFrame with interactions between users and items
    
* `duration_days` - time interval in days
    
* `first` - indicator, if the value is `True`, take first days from log, if the value is `False`, take last days from log
    
* `date_column` - date column name

![title](img/take_num_days_of_global_hist.jpg)

Let's leave the last `3 weeks` of interactions

In [25]:
log_filter = take_num_days_of_global_hist(log_spark, 
                                          duration_days=21,
                                          first=False)

In [27]:
(
    log_filter
    .agg(sf.datediff(sf.max("timestamp"), sf.min("timestamp")).alias("diff"))
    .select(sf.max("diff").alias("Maximum difference in days among all interactions"))
).show()

+-------------------------------------------------+
|Maximum difference in days among all interactions|
+-------------------------------------------------+
|                                               21|
+-------------------------------------------------+



In [28]:
(
    log_spark
    .select(
        sf.max("timestamp")
        .alias("Maximum date in log")
    )
).show()

(
    log_filter
    .select(
        sf.max("timestamp")
        .alias("Maximum date in filtered log")
    )
).show()

+-------------------+
|Maximum date in log|
+-------------------+
|2003-02-28 20:49:50|
+-------------------+

+----------------------------+
|Maximum date in filtered log|
+----------------------------+
|         2003-02-28 20:49:50|
+----------------------------+



After such filtering, the number of interactions and users significantly decreased

In [29]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 920, total users: 82, total items: 704


## function take_time_period()

Select interactions from required time period.

Parameters:
    
* `log` - DataFrame with interactions between users and items
    
* `start_date` - beginning of required period
    
* `end_date` - end of required period
    
* `date_column` - date column name

![title](img/take_time_period.jpg)

Let's leave the data for `March` only

In [30]:
log_filter = take_time_period(log_spark, 
                              start_date="2001-03-01",
                              end_date="2001-04-01")

Check maximum and minimum date

In [31]:
(
    log_filter
    .select(
        sf.min("timestamp").alias("Minimum date"),
        sf.max("timestamp").alias("Maximum date")
    )                            
).show()

+-------------------+-------------------+
|       Minimum date|       Maximum date|
+-------------------+-------------------+
|2001-03-01 03:08:52|2001-03-31 23:12:22|
+-------------------+-------------------+



The number of records has noticeably decreased

In [32]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 5986, total users: 321, total items: 2010
